In [96]:
# import dependencies
import os
import pandas as pd
#import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist 
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, ElementNotInteractableException, JavascriptException
from datetime import date, datetime, timedelta
import holidays
import time
from functions import country_holidays, intHolidayClosures, allMondays, buildBrowser, createFilename

# makes it easier to see dataframes
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
pd.set_option('expand_frame_repr', False)

int_stop_date = date(2023, 9, 30)
dom_stop_date = date(2023, 9, 30)

In [97]:
# function to generate list of trips from
def create_trips(int_stop_date, US):
    for pu_d in allMondays(int_stop_date):
        # drop off date is 17 days after today
        do_d = pu_d + timedelta(17)
        # for all mondays after today (weeks = 36, is the following year)
        if pu_d > date.today() + timedelta(weeks = 52):
            # if monday is a holiday, change pickup to Tuesday
            if pu_d in intHolidayClosures(country_holidays, US):
                pu_d += timedelta(1)
            # if Thursday is a holiday, change drop off to Wednesday
            if do_d in intHolidayClosures(country_holidays, US):
                do_d -= timedelta(1)
            
            yield {'start_date':pu_d.strftime('%Y-%m-%d'),'start_day': pu_d.strftime('%d').lstrip('0'),'start_month': pu_d.strftime('%b'),'start_year': pu_d.strftime('%Y'),'start_date_out':pu_d.strftime('%m-%d-%Y'),
                   'end_date':do_d.strftime('%Y-%m-%d'),'end_day':do_d.strftime('%d').lstrip('0'),'end_month':do_d.strftime('%b'),'end_year':do_d.strftime('%Y')}

# create list to store cities that will be searched
cities = [{'city':'Calgary','state':'AB', 'abbr':'YYC', 'country':'CAN'},
          {'city':'Vancouver','state':'BC', 'abbr':'YVR', 'country':'CAN'},
          {'city':'Montreal','state':'QC', 'abbr':'YUL', 'country':'CAN'},
          {'city':'Toronto','state':'ON', 'abbr':'YYZ', 'country':'CAN'},
          {'city':'Halifax','state':'NS', 'abbr':'YHZ', 'country':'CAN'}]

for city in cities: city['done'] = False

In [98]:
executable_path = {'executable_path':'C:/Users/rbandrowski/AppData/Local/Continuum/anaconda3/chromedriver.exe'}
url = 'http://staging.campertravelcanada.com/'
log_file = createFilename('Camper Travel', True)

In [99]:
click_chat_js = """
                    var button = document.querySelector('button.olark-launch-button.olark-size-md');
                    function eventFire(elem, etype) {
                      if (elem.fireEvent) {
                        elem.fireEvent('on' + etype);
                      } 
                      else {
                        var evObj = document.createEvent('Events');
                        evObj.initEvent(etype, true, false);
                        elem.dispatchEvent(evObj);
                      }
                    }
                    eventFire(button, 'click');
                """

In [100]:
if os.path.exists(log_file):

    with open(log_file, 'r') as fin:
        data = [json.loads(x) for x in fin]

    last_city = data[-1]['Location'].split(',')[0]
    last_date = date[-1]['Pickup Date']

    for city in cities:
        if city['city'] == last_city: break
        else: city['done'] = True

    lt = next((t for t, trip in enumerate(create_trips(dom_stop_date, True)) if trip['pu_date_out'] == last_date))

else:
    lt = -1

In [101]:
deals_df = pd.DataFrame()
# search dates for each city
for c, city in enumerate(cities):

    if not city['done']:
        
        US = True if city['country'] == 'CAN' else False
        browser = buildBrowser(executable_path)
        browser.visit(url)

        # select city for pickup and dropoff inputs
        browser.find_by_xpath("//input[contains(@id, 'pickupLocation')]").click()
        browser.find_by_xpath(f"//div[@class = 'autocomplete-suggestion'][@data-val = '{city['city']}']").click()

        # choose international driver's license
        browser.find_by_css('input.form-control.X-CountryOfResidence.AutoCompleteSelectInput').click()
        browser.find_by_xpath("//div[@class = 'autocomplete-suggestion'][@data-val = 'International']").click()

        

        for t, trip in enumerate(create_trips(int_stop_date, US)):

            if t > lt:
                deals = []
            
                browser.find_by_xpath("//button[contains(@id, 'pickupDate')]").click()

                cal_month = browser.find_by_xpath("//div[@class = 'dr-cal-start']/div/div/div").find_by_css('button.dp-cal-month').text[0:3]

                while cal_month != trip['start_month']:
                    browser.find_by_xpath("//div[@class = 'dr-cal-end']/div/div/div/header/button[@class = 'dp-next']").click()
                    cal_month = browser.find_by_xpath("//div[@class = 'dr-cal-start']/div/div/div").find_by_css('button.dp-cal-month').text[0:3]

                pu_d = browser.find_by_xpath(f"//div[@class = 'dr-cal-start']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['start_day']}]")
                pu_d.click()

                if int(trip['start_day']) < int(trip['end_day']):
                    do_d = browser.find_by_xpath(f"//div[@class = 'dr-cal-start']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['end_day']}]")
                else:
                    do_d = browser.find_by_xpath(f"//div[@class = 'dr-cal-end']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['end_day']}]")
                do_d.click()

                browser.find_by_css('button.btn.btn-success.btn-lg.btn-block.X-SearchButton').click()
                browser.windows.current = browser.windows[1]

                while browser.is_element_not_present_by_css('div.campervan-result'):
                    continue

                #browser.execute_script("window.scrollTo(500, $(document).height());")
                browser.execute_script("window.scrollTo(500, 0);")
                while True:
                    try:
                        browser.execute_script("document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';")
                        break
                    except JavascriptException:
                        continue

                results = browser.find_by_css('div.campervan-result')
                
                for result in results:

                    browser.execute_script("document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';")
                    
                    if len(result.find_by_css('span.X-VPrice-ConvertedFrom-Amount')) > 0:
                    
                        daily_rate = float(result.find_by_css('span.X-VPrice-ConvertedFrom-Amount').text)
                        pu_date = trip['start_date']
                        do_date = trip['end_date']
                        location = f'{city["city"]}, {city["state"]}'

                        if browser.find_by_css('div#olark-container').visible:
                            browser.execute_script(click_chat_js)

                        while True:
                            try:
                                result.find_by_text('Check Availability').click()
                                break
                            except ElementClickInterceptedException:
                                try:
                                    browser.execute_script("window.scrollTo(0, window.scrollY + 200);")
                                except JavascriptException:
                                    continue

                        browser.windows.current = browser.windows[2]
                        start = time.time()
                        while True:
                            try:
                                if time.time() > start + 30:
                                    browser.reload()
                                    # browser.windows.current.close()
                                    # browser.windows.current = browser.windows[1]
                                    # result.find_by_css('a.button-green.button-book').click()
                                    # browser.windows.current = browser.windows[2]
                                    start = time.time()
                            except TimeoutException:
                                continue
                            try:
                                company = browser.find_by_css('img.vehicle-image')['src'].split('/')[-2].replace('-', ' ').lower().rstrip('us').strip().title()
                                print(company)
                                break
                            except ElementDoesNotExist:
                                continue
                        rv_class = browser.find_by_css('div.vehicle-name').text
                        try:
                            section = browser.find_by_css('span.X-VPrice-ConvertedFrom.v-price-total.v-price-converted-from').last
                            total = float(section.find_by_css('span.X-VPrice-ConvertedFrom-Amount').text)
                            print(total)
                        except ElementDoesNotExist:
                            total = 'Not Listed'

                        browser.windows.current.close()
                        browser.windows.current = browser.windows[1]

                        deals.append({'Pickup Date':pu_date,'Dropoff Date':do_date,'Class':rv_class,'Company':company,
                                    'Daily Rate':daily_rate,'Price':total,'Location':location})

                        deals_df = deals_df.append(deals, ignore_index=True)
                
                browser.windows.current.close()

        browser.quit()
        

C:\Users\Kkirkpatrick\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\Kkirkpatrick\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


Cruise America
3113.58
Cruise America
3391.39
Cruise America
3559.99
Cruise America
4013.59
Escape
6359.14
Cruise America
3566.39
Cruise America
3674.99
Cruise America
3603.58
Cruise America
4888.6
Escape
6359.14
Cruise America
4896.38
Cruise America
5314.98
Cruise America
5388.58
Cruise America
5733.58
Escape
6359.14
Escape
5997.71
Canadream Rv
5925.0
Cruise America
6417.33
Canadream Rv
6133.0
Cruise America
6730.13
Canadream Rv
6367.0
Cruise America
6727.33
Escape
6359.14
Cruise America
7278.58
Cruise America
7588.58
Cruise America
8034.98
Cruise America
8091.38
Escape
6359.14
Cruise America
7503.58
Cruise America
7776.38
Cruise America
7728.58
Cruise America
8264.98
Canadream Rv
6198.0
Canadream Rv
6406.0
Cruise America
7095.13
Canadream Rv
6822.0
Cruise America
7152.33
Cruise America
7277.33
Canadream Rv
7666.8
Cruise America
7258.58
Cruise America
7788.58
Cruise America
7996.38
Cruise America
8639.98
Cruise America
7993.58
Cruise America
8358.58
Cruise America
8881.38
Cruise Ameri

KeyboardInterrupt: 

In [102]:
print(deals_df)

     Pickup Date Dropoff Date                       Class         Company  Daily Rate    Price     Location
0     2023-05-08   2023-05-25  C21 Compact Plus Motorhome  Cruise America       96.89  3113.58  Calgary, AB
1     2023-05-08   2023-05-25  C21 Compact Plus Motorhome  Cruise America       96.89  3113.58  Calgary, AB
2     2023-05-08   2023-05-25         C30 Large Motorhome  Cruise America      106.89  3391.39  Calgary, AB
3     2023-05-08   2023-05-25  C21 Compact Plus Motorhome  Cruise America       96.89  3113.58  Calgary, AB
4     2023-05-08   2023-05-25         C30 Large Motorhome  Cruise America      106.89  3391.39  Calgary, AB
...          ...          ...                         ...             ...         ...      ...          ...
2068  2023-09-25   2023-10-12              Run of Fleet 2    Canadream Rv      173.23  4224.52  Toronto, ON
2069  2023-09-25   2023-10-12           Deluxe Van Camper    Canadream Rv      201.40  4703.38  Toronto, ON
2070  2023-09-25   2023-10-1

In [103]:
deals_df.to_csv('camper_travel_data_next_year_can.csv', index=False)